# Test initial Dataloader Change

In [2]:
import pandas as pd
from pathlib import Path
import xarray as xr
import numpy as np
# colormaps = https://matplotlib.org/cmocean/
import cmocean
import matplotlib.pyplot as plt
import geopandas as gpd
from typing import List, Tuple, Dict, Union, Optional

import matplotlib as mpl
import seaborn as sns
mpl.rcParams['figure.dpi'] = 100

import os

# set the working directory
current_path = Path('.').resolve()
if current_path == Path('/home/tommy/ml_drought/notebooks/draft'):
    os.chdir(current_path.parents[1].as_posix())

current_path = Path('.').resolve()
assert current_path.name == "ml_drought"

%load_ext autoreload
%autoreload 2

In [3]:
from scripts.utils import get_data_path
from src.engineer import Engineer
from src.utils import get_ds_mask

In [4]:
data_dir = get_data_path()

# Load the preprocessed data

In [11]:
from src.engineer import Engineer

engineer = Engineer(
    get_data_path(), experiment="one_month_forecast", process_static=True
)
e = engineer.engineer_class

In [12]:
data = e._make_dataset(static=False)

Processing /cats/datastore/data/interim/VCI_preprocessed/data_india.nc
Processing /cats/datastore/data/interim/reanalysis-era5-land-monthly-means_preprocessed/volumetric_soil_water_layer_1_data_india.nc
Processing /cats/datastore/data/interim/reanalysis-era5-land-monthly-means_preprocessed/volumetric_soil_water_layer_2_data_india.nc
Processing /cats/datastore/data/interim/reanalysis-era5-land-monthly-means_preprocessed/volumetric_soil_water_layer_3_data_india.nc
Processing /cats/datastore/data/interim/reanalysis-era5-land-monthly-means_preprocessed/volumetric_soil_water_layer_4_data_india.nc
Processing /cats/datastore/data/interim/reanalysis-era5-land-monthly-means_preprocessed/potential_evaporation_data_india.nc
Processing /cats/datastore/data/interim/reanalysis-era5-land-monthly-means_preprocessed/total_precipitation_data_india.nc
Processing /cats/datastore/data/interim/reanalysis-era5-land-monthly-means_preprocessed/2m_temperature_data_india.nc
Processing /cats/datastore/data/interi

In [14]:
data

<xarray.Dataset>
Dimensions:        (lat: 304, lon: 305, time: 242)
Coordinates:
  * time           (time) datetime64[ns] 2000-02-29 2000-03-31 ... 2020-03-31
  * lon            (lon) float32 67.3 67.4 67.5 67.6 ... 97.4 97.5 97.6 97.7
  * lat            (lat) float32 36.0 35.9 35.8 35.7 35.6 ... 6.0 5.9 5.8 5.7
Data variables:
    VCI            (time, lat, lon) float64 ...
    swvl1          (time, lat, lon) float32 ...
    swvl2          (time, lat, lon) float32 ...
    swvl3          (time, lat, lon) float32 ...
    swvl4          (time, lat, lon) float32 ...
    pev            (time, lat, lon) float32 ...
    tp             (time, lat, lon) float32 ...
    t2m            (time, lat, lon) float32 ...
    e              (time, lat, lon) float32 ...
    t2m_std_1      (time, lat, lon) float64 ...
    t2m_mean_1     (time, lat, lon) float64 ...
    t2m_std_2      (time, lat, lon) float64 ...
    t2m_mean_2     (time, lat, lon) float64 ...
    t2m_std_3      (time, lat, lon) float64 ...
    t2m_mean_3     (time, lat, lon) float64 ...
    erate_std_1    (time, lat, lon) float64 ...
    erate_mean_1   (time, lat, lon) float64 ...
    erate_std_2    (time, lat, lon) float64 ...
    erate_mean_2   (time, lat, lon) float64 ...
    erate_std_3    (time, lat, lon) float64 ...
    erate_mean_3   (time, lat, lon) float64 ...
    tprate_std_1   (time, lat, lon) float64 ...
    tprate_mean_1  (time, lat, lon) float64 ...
    tprate_std_2   (time, lat, lon) float64 ...
    tprate_mean_2  (time, lat, lon) float64 ...
    tprate_std_3   (time, lat, lon) float64 ...
    tprate_mean_3  (time, lat, lon) float64 ...

In [141]:
dynamic_data = data
static_data = e._make_dataset(static=True)

Processing /cats/datastore/data/interim/static/srtm_preprocessed/india.nc
Processing /cats/datastore/data/interim/static/reanalysis-era5-single-levels-monthly-means_preprocessed/data_india.nc
Processing /cats/datastore/data/interim/static/esa_cci_landcover_preprocessed/esa_cci_landcover_india_one_hot.nc


In [133]:
from collections import defaultdict
from typing import DefaultDict, Dict
from pathlib import Path
import xarray as xr


# class Normalizer:
def calculate_normalization_dict(ds: xr.Dataset, static: bool = False) -> DefaultDict[str, Dict[str, float]]:
    normalization_values: DefaultDict[str, Dict[str, float]] = defaultdict(dict)
    if static:
        reducing_dims = ["lat", "lon"]
    else:
        reducing_dims = ["lat", "lon", "time"]

    for var in ds.data_vars:
        if var.endswith("one_hot"):
            mean = 0.0
            std = 1.0
        else:
            mean = float(
                ds[var].mean(dim=reducing_dims, skipna=True).values
            )
            std = float(ds[var].std(
                dim=reducing_dims, skipna=True).values)

        normalization_values[var]["mean"] = mean
        normalization_values[var]["std"] = std
        
    return normalization_values


def normalize_xr(ds: xr.Dataset, static: bool = False) -> Tuple[xr.Dataset, DefaultDict[str, Dict[str, float]]]:
    """Normalize the xarray object

    Args:
        ds (xr.Dataset): [description]
        static (bool, optional): [description]. Defaults to False.

    Returns:
        xr.Dataset: Xarray Dataset with normalized values
        DefaultDict[str, Dict[str, float]]: The values
    """
    norm_dict = calculate_normalization_dict(ds, static)

    list_of_normed: List[xr.DataArray] = []
    for variable in ds.data_vars:
        list_of_normed.append((ds[variable] - norm_dict[variable]
                        ["mean"]) / norm_dict[variable]["std"])

    ds_norm = xr.merge(list_of_normed)

    return ds_norm, norm_dict


def unnormalize_xr(ds: xr.Dataset, normalization_dict: DefaultDict[str, Dict[str, float]]) -> xr.Dataset:
    list_of_unnormed: List[xr.DataArray] = []
    for variable in ds.data_vars:
        list_of_unnormed.append(
            (ds[variable] * normalization_dict[variable]["std"]) +
            normalization_dict[variable]["mean"]
        )

    ds_unnorm = xr.merge(list_of_unnormed)

    return ds_unnorm

In [139]:
cfg = dict(
    dynamic_inputs=["tp", "tprate_mean_1"],
    target_variable=["VCI"],
    static_inputs=[],
    seq_length=3,
    min_test_time="2010-01-01",
    max_test_time="2020-01-01",
    min_train_time="2000-01-01",
    max_train_time="2009-12-31",
)

test_times = pd.date_range(cfg["min_test_time"], cfg["max_test_time"], freq="M")
train_times = pd.date_range(cfg["min_train_time"], cfg["max_train_time"], freq="M")

In [143]:
dyn_scaler = calculate_normalization_dict(dynamic_data)
stat_scaler = calculate_normalization_dict(static_data, static=True)

# Convert into pixel based information

In [16]:
pixels = data.copy().stack(pixel=["lat", "lon"])

In [19]:
df = data.to_dataframe()

In [20]:
df.head()

VCI     swvl1     swvl2     swvl3     swvl4  \
lat  lon       time                                                            
36.0 67.300003 2000-02-29  63.625000  0.414532  0.389866  0.294564  0.308208   
               2000-03-31  55.457497  0.402257  0.393125  0.327063  0.308119   
               2000-04-30  47.074997  0.319747  0.325866  0.323359  0.307672   
               2000-05-31  11.894000  0.217281  0.246682  0.292880  0.305913   
               2000-06-30   2.427500  0.204540  0.228852  0.263577  0.303094   

                                pev        tp         t2m         e  \
lat  lon       time                                                   
36.0 67.300003 2000-02-29 -0.000927  0.000686  269.374756 -0.000200   
               2000-03-31 -0.001773  0.000501  275.354950 -0.000781   
               2000-04-30 -0.003480  0.000563  285.641968 -0.001639   
               2000-05-31 -0.004650  0.000050  291.656830 -0.001123   
               2000-06-30 -0.004817  0.000083  292.572998 -0.000602   

                           t2m_std_1  ...   erate_std_2  erate_mean_2  \
lat  lon       time                   ...                               
36.0 67.300003 2000-02-29   1.683767  ...           NaN           NaN   
               2000-03-31   1.541664  ...  1.035139e-09 -3.085186e-09   
               2000-04-30   1.380861  ...  3.013228e-09 -8.664283e-09   
               2000-05-31   1.355013  ...  3.054498e-09 -2.387456e-08   
               2000-06-30   0.817621  ...  4.978621e-09 -3.039982e-08   

                            erate_std_3  erate_mean_3  tprate_std_1  \
lat  lon       time                                                   
36.0 67.300003 2000-02-29           NaN           NaN  4.392460e-09   
               2000-03-31           NaN           NaN  3.034482e-09   
               2000-04-30  2.584970e-09 -8.769980e-09  4.763290e-09   
               2000-05-31  4.536804e-09 -2.057241e-08  9.622732e-09   
               2000-06-30  2.819519e-09 -3.373821e-08  1.482762e-09   

                           tprate_mean_1  tprate_std_2  tprate_mean_2  \
lat  lon       time                                                     
36.0 67.300003 2000-02-29   1.364650e-08           NaN            NaN   
               2000-03-31   1.347471e-08  4.559223e-09   1.636154e-08   
               2000-04-30   1.997592e-08  9.146565e-09   2.593243e-08   
               2000-05-31   2.031139e-08  8.299525e-09   2.892003e-08   
               2000-06-30   4.693968e-09  8.613428e-09   1.330562e-08   

                           tprate_std_3  tprate_mean_3  
lat  lon       time                                     
36.0 67.300003 2000-02-29           NaN            NaN  
               2000-03-31           NaN            NaN  
               2000-04-30  9.685661e-09   2.447746e-08  
               2000-05-31  8.969445e-09   3.169543e-08  
               2000-06-30  1.054342e-08   1.819778e-08  

[5 rows x 27 columns]

In [21]:
df.shape

(22438240, 27)

In [27]:
d = df.iloc[0:1000]

In [32]:
metadata = d.index.to_numpy()


In [53]:
def get_pixel_data(ds, pixel):
    pass


NameError: name 'input_variables' is not defined

In [ ]:
# import h5py

# h5_file = data_dir / "test.h5"

# n_dyn_inputs = len(cfg["dynamic_inputs"])
# n_targets = len(cfg["target_variable"])
# # we only store user-defined additional static features provided in the additional_features table
# n_stat = len(cfg["static_inputs"])

# with h5py.File(h5_file, "w") as out_f:
#     dyn_input_data = out_f.create_dataset(
#         "dynamic_inputs",
#         shape=(0, cfg["seq_length"], n_dyn_inputs),
#         maxshape=(None, cfg["seq_length"], n_dyn_inputs),
#         chunks=True,
#         dtype=np.float32,
#         compression="gzip",
#     )
#     if n_stat > 0:
#         stat_input_data = out_f.create_dataset(
#             "static_inputs",
#             shape=(0, n_stat),
#             maxshape=(None, n_stat),
#             chunks=True,
#             dtype=np.float32,
#             compression="gzip",
#         )
#     target_data = out_f.create_dataset(
#         "target_data",
#         shape=(0, cfg["seq_length"], n_targets),
#         maxshape=(None, cfg["seq_length"], n_targets),
#         chunks=True,
#         dtype=np.float32,
#         compression="gzip",
#     )

#     sample_2_basin = out_f.create_dataset(
#         "sample_2_basin",
#         shape=(0,),
#         maxshape=(None,),
#         dtype="S11",
#         compression="gzip",
#         chunks=True,
#     )

In [62]:
input_variables = ["tp", "tprate_mean_1"]
target_variable = "VCI"

In [63]:
# sample_id
for spatial_unit in all_spatial_units:
    pass

In [64]:
all_spatial_units = data.stack(pixel=["lat", "lon"]).pixel.values[:5]

# TEST
spatial_unit = all_spatial_units[0]
# ------

su_data = data.sel(lat=spatial_unit[0], lon=spatial_unit[1])

In [146]:
total_samples = 0
spatial_units_without_train_data = []

dynamic_data = su_data[cfg["dynamic_inputs"] + cfg["target_variable"]]

# drop missing timesteps
missing_timesteps = np.any(np.isnan(dynamic_data).to_array().values, axis=0)
assert len(missing_timesteps.shape) == 1, "Expect to have reduced missing data shape to missing timesteps (in any variable) (1 Dimensional)"
dynamic_data = dynamic_data.sel(time=~missing_timesteps)

# update data in h5 file
num_samples = len(dynamic_data["time"])
total_samples = dyn_input_data.shape[0] + num_samples

# resize h5file
# dyn_input_data.resize((total_samples, cfg["seq_length"], n_dyn_inputs))




ValueError: Not a dataset (not a dataset)

NameError: name 'norm_dict' is not defined

In [125]:
# import h5py

# with h5py.File(h5_file, "w") as out_f:
#     dyn_input_data = out_f.create_dataset(
#         "dynamic_inputs",
#         shape=(0, cfg["seq_length"], n_dyn_inputs),
#         maxshape=(None, cfg["seq_length"], n_dyn_inputs),
#         chunks=True,
#         dtype=np.float32,
#         compression="gzip",
#     )
#     if n_stat > 0:
#         stat_input_data = out_f.create_dataset(
#             "static_inputs",
#             shape=(0, n_stat),
#             maxshape=(None, n_stat),
#             chunks=True,
#             dtype=np.float32,
#             compression="gzip",
#         )
#     target_data = out_f.create_dataset(
#         "target_data",
#         shape=(0, cfg["seq_length"], n_targets),
#         maxshape=(None, cfg["seq_length"], n_targets),
#         chunks=True,
#         dtype=np.float32,
#         compression="gzip",
#     )
#     q_stds = out_f.create_dataset(
#         "q_stds",
#         shape=(0, 1),
#         maxshape=(None, 1),
#         dtype=np.float32,
#         compression="gzip",
#         chunks=True,
#     )
#     sample_2_basin = out_f.create_dataset(
#         "sample_2_basin",
#         shape=(0,),
#         maxshape=(None,),
#         dtype="S11",
#         compression="gzip",
#         chunks=True,
#     )

#     scalers = {
#         "dyn_mean": np.zeros(n_dyn_inputs),
#         "dyn_std": np.zeros(n_dyn_inputs),
#         "target_mean": np.zeros(n_targets),
#         "target_std": np.zeros(n_targets),
#     }
#     total_samples = 0

#     basins_without_train_data = []


In [86]:
dynamic_data.where(np.isna)

TypeError: isnull() got an unexpected keyword argument 'dim'

In [99]:
np.any(np.isnan(dynamic_data).to_array().values, axis=-1)

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

3

<xarray.Dataset>
Dimensions:        (time: 242)
Coordinates:
  * time           (time) datetime64[ns] 2000-02-29 2000-03-31 ... 2020-03-31
    lon            float32 67.3
    lat            float32 36.0
Data variables:
    VCI            (time) float64 63.62 55.46 47.07 11.89 ... 77.29 64.9 36.12
    swvl1          (time) float32 0.41453198 0.40225673 ... 0.42658207 0.4277759
    swvl2          (time) float32 0.38986608 0.39312515 ... 0.41466212
    swvl3          (time) float32 0.29456416 0.32706335 ... 0.32518026
    swvl4          (time) float32 0.30820844 0.30811885 ... 0.28468922
    pev            (time) float32 -0.00092692673 -0.0017727639 ... -0.0016514888
    tp             (time) float32 0.0006858027 0.0005013043 ... 0.00144986
    t2m            (time) float32 269.37476 275.35495 ... 273.01175 277.6138
    e              (time) float32 -0.00020003476 ... -0.00092126103
    t2m_std_1      (time) float64 1.684 1.542 1.381 1.355 ... 1.078 nan nan
    t2m_mean_1     (time) float64 264.9 265.0 269.5 280.8 ... 268.6 nan nan
    t2m_std_2      (time) float64 nan 2.177 2.6 1.712 ... 1.776 2.179 2.349 nan
    t2m_mean_2     (time) float64 nan 264.2 269.8 279.3 ... 267.0 263.4 nan
    t2m_std_3      (time) float64 nan nan 2.579 2.47 ... 2.468 2.679 1.692 2.205
    t2m_mean_3     (time) float64 nan nan 270.6 277.6 ... 266.4 262.6 265.6
    erate_std_1    (time) float64 3.304e-10 1.076e-09 1.504e-09 ... nan nan
    erate_mean_1   (time) float64 -2.664e-09 -4.111e-09 -9.503e-09 ... nan nan
    erate_std_2    (time) float64 nan 1.035e-09 3.013e-09 ... 5.209e-10 nan
    erate_mean_2   (time) float64 nan -3.085e-09 -8.664e-09 ... -1.958e-09 nan
    erate_std_3    (time) float64 nan nan 2.585e-09 ... 4.535e-10 1.062e-09
    erate_mean_3   (time) float64 nan nan -8.77e-09 ... -1.762e-09 -3.305e-09
    tprate_std_1   (time) float64 4.392e-09 3.034e-09 4.763e-09 ... nan nan
    tprate_mean_1  (time) float64 1.365e-08 1.347e-08 1.998e-08 ... nan nan
    tprate_std_2   (time) float64 nan 4.559e-09 9.147e-09 ... 4.399e-09 nan
    tprate_mean_2  (time) float64 nan 1.636e-08 2.593e-08 ... 1.44e-08 nan
    tprate_std_3   (time) float64 nan nan 9.686e-09 ... 5.28e-09 4.584e-09
    tprate_mean_3  (time) float64 nan nan 2.448e-08 ... 1.204e-08 1.506e-08

(92720,)

In [24]:
304*305*242

22438240